In [4]:


import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
import lightgbm as lgb
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
#from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb


In [24]:

train_set = pd.read_csv(r'C:\Users\hrahe\Downloads\train_set.csv')
test_set = pd.read_csv(r'C:\Users\hrahe\Downloads\test_set.csv')

In [25]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [26]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [27]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [6]:
# scalar=MinMaxScaler()
# X = scalar.fit_transform(X)
# X_testdata = scalar.transform(X_testdata)




In [7]:
# import lightgbm as lgb
# import pandas as pd
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import train_test_split


# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# # Initialize and train the LightGBM model on full feature set to get feature importances
# clf = lgb.LGBMClassifier(
#     objective='binary',
#     metric='binary_logloss',
#     boosting_type='gbdt',
#     num_leaves=20,
#     max_depth=4,
#     learning_rate=0.03,
#     n_estimators=376,
#     subsample=0.7,
#     colsample_bytree=0.5,
#     min_child_weight=5,
#     scale_pos_weight=1.1,
#     reg_lambda=0.1,
#     verbose=-1
# )

# # Fit the model on full data to get feature importances
# clf.fit(X_train, y_train)

# # Get feature importances and create a DataFrame
# feature_importances = clf.feature_importances_
# feature_importances_df = pd.DataFrame({
#     'Feature': X_train.columns,
#     'Importance': feature_importances
# })

# # Sort the DataFrame by importance
# feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# # Define different numbers of top features to test
# feature_counts = [10, 15, 20, 25, 30,35,40,45,50,55,60]
# auc_scores = {}

# # Iterate over different numbers of top features
# for n in feature_counts:
#     # Select the top n features
#     top_features = feature_importances_df['Feature'].head(n).tolist()

#     # Subset the training and testing data to the top n features
#     X_train_top = X_train[top_features]
#     X_test_top = X_test[top_features]

#     # Re-train the model with the top n features
#     clf.fit(X_train_top, y_train)

#     # Predict probabilities on the test set
#     y_pred_proba = clf.predict_proba(X_test_top)[:, 1]

#     # Calculate AUC score
#     auc = roc_auc_score(y_test, y_pred_proba)
#     auc_scores[n] = auc  # Store the AUC score for this number of features

#     print(f"Top {n} features - AUC: {auc:.4f}")

# # Print all AUC scores for comparison
# print("\nAUC scores for different feature counts:")
# for n, auc in auc_scores.items():
#     print(f"Top {n} features: AUC = {auc:.4f}")


In [29]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.24, random_state=1)

In [9]:
# from sklearn.model_selection import RandomizedSearchCV
# import xgboost as xgb

# # Define the parameter grid
# param_grid = {
#     'learning_rate': [0.01, 0.03, 0.1],
#     'max_depth': [4, 6, 8],
#     'n_estimators': [100, 200, 376,500],
#     'gamma': [0, 0.1, 0.2],
#     'reg_lambda': [0.01, 0.1, 1],
#     'min_child_weight': [1, 3, 5],
#     'colsample_bytree': [0.5, 0.7, 1.0],
#     'colsample_bylevel': [0.5, 0.7, 1.0],
#     'scale_pos_weight': [1, 1.1, 1.2],
#     'subsample': [0.7, 0.8, 1.0]
# }

# # Initialize the classifier
# clf = xgb.XGBClassifier(
#     objective='binary:logistic',
#     eval_metric='logloss',
#     random_state=42
# )

# random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_grid, cv=3, n_iter=50, scoring='roc_auc', verbose=1, n_jobs=-1)

# # Fit the model
# random_search.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters found: ", random_search.best_params_)

In [10]:
# # prompt: like the following geneerate good params for catboost but stire separately:import xgboost as xgb
# # from sklearn.model_selection import GridSearchCV
# # # Define the parameter grid
# # param_grid = {
# #     'learning_rate': [0.01, 0.03, 0.1],
# #     'max_depth': [4, 6, 8],
# #     'n_estimators': [100, 200, 376,500],
# #     'gamma': [0, 0.1, 0.2],
# #     'reg_lambda': [0.01, 0.1, 1],
# #     'min_child_weight': [1, 3, 5],
# #     'colsample_bytree': [0.5, 0.7, 1.0],
# #     'colsample_bylevel': [0.5, 0.7, 1.0],
# #     'scale_pos_weight': [1, 1.1, 1.2],
# #     'subsample': [0.7, 0.8, 1.0]
# # }
# # # Initialize the classifier
# # clf = xgb.XGBCla

# from catboost import CatBoostClassifier
# from sklearn.model_selection import train_test_split, GridSearchCV
# import json

# # Assuming X_train, y_train are defined from your previous code

# # Define the parameter grid for CatBoost
# param_grid = {
#     'iterations': [100, 200, 300],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'depth': [4, 6, 8],
#     'l2_leaf_reg': [1, 3, 5],
#     'border_count': [32, 64, 128],
#     'random_strength': [1, 1.5, 2],
#     'bagging_temperature': [0, 1, 2],
# }

# # Initialize the CatBoost classifier
# clf2 = CatBoostClassifier(
#     loss_function='Logloss',
#     eval_metric='AUC',
#     task_type='GPU',  # Use GPU if available
#     random_seed=42
# )


# # Initialize GridSearchCV for CatBoost
# grid_search2 = GridSearchCV(estimator=clf2, param_grid=param_grid, cv=3, scoring='roc_auc', verbose=1, n_jobs=-1)

# # Fit the model
# grid_search2.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters found: ", grid_search2.best_params_)

# # Store the best parameters in a JSON file
# best_params2 = grid_search2.best_params_
# with open('best_catboost_params.json', 'w') as f:
#     json.dump(best_params2, f, indent=4)

In [11]:
# from sklearn.model_selection import RandomizedSearchCV


# # Define the parameter grid for LightGBM
# param_grid = {
#     'learning_rate': [0.01, 0.03, 0.1],
#     'max_depth': [4, 6, 8, -1],  # -1 indicates no limit for depth in LightGBM
#     'n_estimators': [100, 200, 376, 500],
#     'num_leaves': [20, 31, 40, 50],  # Controls complexity in LightGBM
#     'reg_lambda': [0.01, 0.1, 1],
#     'min_child_weight': [1, 3, 5],
#     'colsample_bytree': [0.5, 0.7, 1.0],
#     'subsample': [0.7, 0.8, 1.0],
#     'scale_pos_weight': [1, 1.1, 1.2]
# }

# # Initialize the classifier
# clf = lgb.LGBMClassifier(
#     objective='binary',
#     random_state=42
# )

# random_search = RandomizedSearchCV(
#     estimator=clf,
#     param_distributions=param_grid,
#     cv=3,
#     n_iter=50,
#     scoring='roc_auc',
#     verbose=1,
#     n_jobs=-1
# )

# # Fit the model
# random_search.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters found: ", random_search.best_params_)


In [12]:
# clf =xgb.XGBClassifier(
#       subsample=1.0,
#       scale_pos_weight= 1.2,
#       reg_lambda= 0.01,
#       n_estimators= 500,
#       min_child_weight= 5,
#       max_depth= 6,
#       learning_rate= 0.03,
#       gamma= 0.2,
#       colsample_bytree= 0.7,
#       colsample_bylevel=0.7,
#       objective='binary:logistic',
#     eval_metric='logloss',
#     random_state=42

# )

# clf.fit(X_train, y_train)

In [13]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define a smaller parameter grid for AdaBoost
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1.0],
    'estimator__max_depth': [1, 2, 3],
    'estimator__min_samples_split': [2, 5]
}

# Initialize the AdaBoost classifier with a DecisionTree as the base estimator
model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(random_state=42),
    random_state=42
)

# Set up RandomizedSearchCV with fewer iterations and cross-validation folds
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    cv=2,  # Reduced from 3 to 2
    n_iter=20,  # Reduced from 50 to 20
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1
)

# Fit the model
random_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", random_search.best_params_)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
Best parameters found:  {'n_estimators': 100, 'learning_rate': 1.0, 'estimator__min_samples_split': 2, 'estimator__max_depth': 1}


In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# Initialize a new AdaBoost model with simplified parameters
clf = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(
        max_depth=4,  # Reduced max_depth
        min_samples_split=5,  # Increased min_samples_split
        random_state=42
    ),
    n_estimators=120,  # Reduced number of estimators
    learning_rate=0.01,
    random_state=42
)

# Fit the model with the best parameters on the training data
clf.fit(X_train, y_train)

# Optionally, evaluate the model on the training or validation set
print("Model trained with simplified parameters.")

c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Model trained with simplified parameters.


In [ ]:

clf.fit(X_train, y_train)
# # Initialize the LightGBM model
# clf = lgb.LGBMClassifier(  objective= 'binary',
#     metric= 'binary_logloss',
#     boosting_type= 'gbdt',
#     num_leaves= 20,
#     max_depth= 5,
#     learning_rate= 0.03,
#     n_estimators= 500,
#     subsample= 0.7,
#     colsample_bytree= 0.5,
#     min_child_weight= 5,
#     scale_pos_weight= 1.1,
#     reg_lambda= 0.1,
#     verbose= -1)

# clf.fit(X_train, y_train)

c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         min_samples_split=5,
                                                         random_state=42),
                   learning_rate=0.01, n_estimators=100, random_state=42)

In [ ]:
# from sklearn.feature_selection import SequentialFeatureSelector

# # Initialize Sequential Feature Selector
# sfs = SequentialFeatureSelector(
#     clf,
#     direction='forward',
#     n_features_to_select=10,
#     scoring='roc_auc',
#     cv=5  # Cross-validation
# )

# # Fit the Sequential Feature Selector
# sfs.fit(X_train, y_train)

# # Get selected feature indices and names
# selected_features = sfs.get_support(indices=True)
# selected_feature_names = X_train.columns[selected_features]

# print("Selected Features:", selected_feature_names.tolist())

# top_features = selected_feature_names.tolist()

In [35]:

feature_importances = clf.feature_importances_

# Create a DataFrame for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
print(feature_importances_df)

#top_features = feature_importances_df['Feature'].head(50).tolist()
top_features_2=feature_importances_df['Feature'].head(30).tolist()
top_features=top_features_2
##20-->  0.9419342388068738


ValueError: All arrays must be of the same length

In [ ]:

# from sklearn.ensemble import RandomForestClassifier  # Or your preferred classifier
# import pandas as pd


# # Initialize variables
# selected_features = []
# remaining_features = X_train.columns.tolist()
# top_features = []  # Store selected top features
# max_features = 15  # Define the maximum number of features to select

# # Perform forward selection using AUC-ROC as the scoring metric
# for i in range(max_features):
#     scores = []
#     # Test each remaining feature by adding it to the selected_features
#     for feature in remaining_features:
#         # Define the current set of features
#         current_features = selected_features + [feature]

#         # Perform cross-validation using only the current set of features
#         score = cross_val_score(clf, X_train[current_features], y_train, cv=5, scoring='roc_auc').mean()
#         scores.append((feature, score))

#     # Find the feature with the best score
#     best_feature, best_feature_score = max(scores, key=lambda x: x[1])

#     # If we still want to select features, update selected_features
#     selected_features.append(best_feature)
#     remaining_features.remove(best_feature)
#     top_features.append(best_feature)

# # Output the selected top features
# print("Top selected features:", top_features)


In [32]:
X_train_top = X_train[top_features]
X_test_top = X_test[top_features]
X_testdata_top = X_testdata[top_features]

# Re-train the model with the top features
clf.fit(X_train_top, y_train)

c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=4,
                                                         min_samples_split=5,
                                                         random_state=42),
                   learning_rate=0.01, n_estimators=120, random_state=42)

In [33]:
y_pred_proba = clf.predict_proba(X_test_top)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc}")

AUC: 0.9383578113572223


In [15]:
clf.fit(X, y)

c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                         min_samples_split=5,
                                                         random_state=42),
                   learning_rate=0.01, n_estimators=100, random_state=42)

In [34]:
# Predict probabilities for the test set
y_pred_proba_testdata = clf.predict_proba(X_testdata_top)[:, 1]

# Create a DataFrame with RecordId and predicted probabilities
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

Probabilities saved to test_set_with_probabilities.csv
[0.00914844 0.01893653 0.00622982 ... 0.02529458 0.00837391 0.00919426]
        RecordId  Y_probability
0         300001       0.009148
1         300002       0.018937
2         300003       0.006230
3         300004       0.010837
4         300005       0.011101
...          ...            ...
105477    405478       0.006407
105478    405479       0.221385
105479    405480       0.016206
105480    405481       0.015919
105481    405482       0.006729

[105482 rows x 2 columns]
